# Día 5: Testing Unitario con pytest

## Descripción General

El testing unitario es una práctica fundamental en el desarrollo de software que consiste en probar componentes individuales de código de forma aislada. pytest es el framework de testing más popular en Python, conocido por su sintaxis simple, características poderosas y extensibilidad.

En este notebook aprenderás a escribir tests efectivos usando pytest, incluyendo fixtures para configuración de tests, mocking para aislar dependencias, y cómo medir la cobertura de código para asegurar que tu código está bien probado.

## Objetivos de Aprendizaje

Al finalizar este notebook, serás capaz de:

1. Escribir tests unitarios efectivos usando pytest con aserciones claras
2. Utilizar fixtures de pytest para configurar y limpiar el estado de los tests
3. Aplicar mocking para aislar dependencias externas en tus tests
4. Medir y mejorar la cobertura de código de tus tests
5. Organizar y estructurar suites de tests de manera profesional

## 1. Introducción a pytest

### ¿Qué es pytest?

pytest es un framework de testing para Python que hace que escribir tests sea simple y escalable. A diferencia de unittest (el framework incluido en la biblioteca estándar), pytest usa aserciones simples de Python y descubre tests automáticamente.

### El Problema que Resuelve

Sin un framework de testing robusto, los desarrolladores enfrentan:
- **Código frágil**: Cambios pequeños rompen funcionalidad existente
- **Regresiones**: Bugs que ya fueron arreglados vuelven a aparecer
- **Miedo a refactorizar**: Sin tests, modificar código es arriesgado
- **Debugging lento**: Encontrar bugs en producción es costoso

### Aprendizaje Clave

pytest simplifica el testing con aserciones de Python estándar y descubrimiento automático de tests. No necesitas heredar de clases especiales ni usar métodos assert especiales.

**Referencia oficial:** [pytest Documentation](https://docs.pytest.org/en/stable/)

In [ ]:
# BAD: Using unittest requires boilerplate
import unittest

class TestCalculator(unittest.TestCase):
    def test_add(self):
        self.assertEqual(2 + 2, 4)

# GOOD: pytest is simple and clean
def test_add():
    assert 2 + 2 == 4

## 2. Escribiendo Tests Básicos

### Estructura de un Test

Un buen test sigue el patrón AAA (Arrange-Act-Assert):
1. **Arrange**: Configurar el estado necesario
2. **Act**: Ejecutar la función bajo prueba
3. **Assert**: Verificar el resultado

### Aprendizaje Clave

Los tests deben ser independientes, repetibles y rápidos. Cada test debe probar una sola cosa y tener un nombre descriptivo que explique qué está probando.

**Referencia oficial:** [pytest: How to write and report assertions](https://docs.pytest.org/en/stable/how-to/assert.html)

In [ ]:
# Example function to test
def calculate_discount(price: float, discount_percent: float) -> float:
    """
    Calculate the final price after applying a discount.
    
    :param price: Original price
    :type price: float
    :param discount_percent: Discount percentage (0-100)
    :type discount_percent: float
    :return: Final price after discount
    :rtype: float
    """
    if discount_percent < 0 or discount_percent > 100:
        raise ValueError("Discount must be between 0 and 100")
    return price * (1 - discount_percent / 100)

# Test following AAA pattern
def test_calculate_discount_normal_case():
    # Arrange
    price = 100.0
    discount = 20.0
    
    # Act
    result = calculate_discount(price, discount)
    
    # Assert
    assert result == 80.0

def test_calculate_discount_zero_discount():
    assert calculate_discount(100.0, 0.0) == 100.0

def test_calculate_discount_full_discount():
    assert calculate_discount(100.0, 100.0) == 0.0

### Pregunta de Comprensión

¿Por qué es importante que cada test sea independiente de los demás?

## 3. Testing de Excepciones

### Verificando que se Lancen Excepciones

Es tan importante probar que tu código falla correctamente como probar que funciona correctamente. pytest proporciona `pytest.raises()` para verificar excepciones.

### Aprendizaje Clave

Usa `pytest.raises()` como context manager para verificar que se lance la excepción correcta con el mensaje apropiado. Esto asegura que tu código maneja errores de forma predecible.

**Referencia oficial:** [pytest: Assertions about expected exceptions](https://docs.pytest.org/en/stable/how-to/assert.html#assertions-about-expected-exceptions)

In [ ]:
import pytest

def test_calculate_discount_invalid_negative():
    with pytest.raises(ValueError) as exc_info:
        calculate_discount(100.0, -10.0)
    
    assert "between 0 and 100" in str(exc_info.value)

def test_calculate_discount_invalid_over_100():
    with pytest.raises(ValueError):
        calculate_discount(100.0, 150.0)

## 4. Fixtures en pytest

### ¿Qué son las Fixtures?

Las fixtures son funciones que se ejecutan antes de los tests para configurar el estado necesario. Son una forma de compartir código de configuración entre múltiples tests sin repetición.

### El Problema que Resuelve

Sin fixtures, terminas duplicando código de configuración en cada test, violando el principio DRY y haciendo los tests difíciles de mantener.

### Aprendizaje Clave

Las fixtures de pytest se declaran con el decorador `@pytest.fixture` y se inyectan en tests simplemente añadiéndolas como parámetros. pytest maneja automáticamente la configuración y limpieza.

**Referencia oficial:** [pytest: How to use fixtures](https://docs.pytest.org/en/stable/how-to/fixtures.html)

In [ ]:
import pytest
from typing import List

class ShoppingCart:
    """
    A simple shopping cart implementation.
    """
    def __init__(self):
        self.items: List[dict] = []
    
    def add_item(self, name: str, price: float, quantity: int = 1) -> None:
        """
        Add an item to the cart.
        
        :param name: Item name
        :type name: str
        :param price: Item price
        :type price: float
        :param quantity: Item quantity
        :type quantity: int
        """
        self.items.append({"name": name, "price": price, "quantity": quantity})
    
    def total(self) -> float:
        """
        Calculate total cart value.
        
        :return: Total price
        :rtype: float
        """
        return sum(item["price"] * item["quantity"] for item in self.items)

# BAD: Duplicating setup code
def test_add_item_bad():
    cart = ShoppingCart()
    cart.add_item("Apple", 1.0)
    assert len(cart.items) == 1

def test_total_bad():
    cart = ShoppingCart()  # Duplicated!
    cart.add_item("Apple", 1.0, 2)
    assert cart.total() == 2.0

# GOOD: Using fixtures
@pytest.fixture
def empty_cart():
    """Provide an empty shopping cart."""
    return ShoppingCart()

@pytest.fixture
def cart_with_items():
    """Provide a cart with sample items."""
    cart = ShoppingCart()
    cart.add_item("Apple", 1.0, 2)
    cart.add_item("Banana", 0.5, 3)
    return cart

def test_add_item_good(empty_cart):
    empty_cart.add_item("Apple", 1.0)
    assert len(empty_cart.items) == 1

def test_total_good(cart_with_items):
    # Cart already has items: 2 apples ($1 each) + 3 bananas ($0.5 each)
    assert cart_with_items.total() == 3.5

### Pregunta de Comprensión

¿Cuál es la ventaja principal de usar fixtures en lugar de duplicar código de configuración?

## 5. Fixture Scopes y Teardown

### Controlando el Ciclo de Vida de las Fixtures

Las fixtures pueden tener diferentes scopes que controlan cuándo se crean y destruyen:
- **function** (default): Una nueva instancia por cada test
- **class**: Una instancia compartida por todos los tests en una clase
- **module**: Una instancia compartida por todos los tests en un módulo
- **session**: Una instancia compartida por toda la sesión de tests

### Aprendizaje Clave

Usa `yield` en fixtures para separar setup y teardown. El código antes de `yield` es setup, el código después es teardown. Esto garantiza limpieza incluso si el test falla.

**Referencia oficial:** [pytest: Fixture scopes](https://docs.pytest.org/en/stable/how-to/fixtures.html#scope-sharing-fixtures-across-classes-modules-packages-or-session)

In [ ]:
import tempfile
import os

# Fixture with teardown using yield
@pytest.fixture
def temp_file():
    """
    Create a temporary file for testing.
    Automatically cleans up after the test.
    """
    # Setup: create temp file
    fd, path = tempfile.mkstemp()
    os.close(fd)
    
    # Provide the path to the test
    yield path
    
    # Teardown: remove temp file
    if os.path.exists(path):
        os.remove(path)

def test_file_operations(temp_file):
    # Write to file
    with open(temp_file, 'w') as f:
        f.write("test data")
    
    # Read from file
    with open(temp_file, 'r') as f:
        content = f.read()
    
    assert content == "test data"
    # File will be automatically deleted after this test

# Fixture with module scope (created once per module)
@pytest.fixture(scope="module")
def database_connection():
    """
    Simulate a database connection.
    Created once for all tests in the module.
    """
    print("\nConnecting to database...")
    connection = {"connected": True, "data": []}
    
    yield connection
    
    print("\nClosing database connection...")
    connection["connected"] = False

## 6. Mocking con pytest

### ¿Qué es Mocking?

Mocking es la técnica de reemplazar dependencias reales con objetos simulados (mocks) durante los tests. Esto permite:
- Aislar el código bajo prueba
- Evitar llamadas a APIs externas, bases de datos, o sistemas de archivos
- Simular condiciones difíciles de reproducir (errores de red, timeouts)
- Hacer tests más rápidos y confiables

### El Problema que Resuelve

Sin mocking, tus tests dependen de recursos externos que pueden:
- Ser lentos (llamadas HTTP, consultas a BD)
- Ser no deterministas (APIs que cambian)
- Requerir configuración compleja
- Fallar por razones ajenas a tu código

### Aprendizaje Clave

Usa `unittest.mock` (incluido en Python) o `pytest-mock` para crear mocks. Los mocks te permiten verificar que tu código llama a las dependencias correctamente sin ejecutarlas realmente.

**Referencia oficial:** [unittest.mock Documentation](https://docs.python.org/3/library/unittest.mock.html)

In [ ]:
from unittest.mock import Mock, patch
import requests

def fetch_user_data(user_id: int) -> dict:
    """
    Fetch user data from an external API.
    
    :param user_id: User ID to fetch
    :type user_id: int
    :return: User data dictionary
    :rtype: dict
    """
    response = requests.get(f"https://api.example.com/users/{user_id}")
    response.raise_for_status()
    return response.json()

# BAD: Testing without mocking (makes real HTTP request)
def test_fetch_user_data_bad():
    # This would make a real HTTP request!
    # Slow, unreliable, requires internet
    pass

# GOOD: Testing with mocking
@patch('requests.get')
def test_fetch_user_data_success(mock_get):
    # Arrange: configure the mock
    mock_response = Mock()
    mock_response.json.return_value = {"id": 1, "name": "John Doe"}
    mock_response.raise_for_status.return_value = None
    mock_get.return_value = mock_response
    
    # Act: call the function
    result = fetch_user_data(1)
    
    # Assert: verify the result and that the mock was called correctly
    assert result == {"id": 1, "name": "John Doe"}
    mock_get.assert_called_once_with("https://api.example.com/users/1")

@patch('requests.get')
def test_fetch_user_data_error(mock_get):
    # Simulate an HTTP error
    mock_get.side_effect = requests.HTTPError("404 Not Found")
    
    with pytest.raises(requests.HTTPError):
        fetch_user_data(999)

### Pregunta de Comprensión

¿Por qué es importante usar mocking cuando se prueban funciones que hacen llamadas HTTP?

## 7. Cobertura de Código (Code Coverage)

### ¿Qué es la Cobertura de Código?

La cobertura de código mide qué porcentaje de tu código es ejecutado por tus tests. Es una métrica útil para identificar código no probado, pero no garantiza que los tests sean buenos.

### Midiendo Cobertura con pytest-cov

pytest-cov es un plugin que integra coverage.py con pytest, permitiendo medir cobertura fácilmente.

### Aprendizaje Clave

Una cobertura del 100% no significa que tu código esté libre de bugs, solo que todas las líneas fueron ejecutadas. Lo importante es la calidad de los tests, no solo la cantidad de líneas cubiertas.

**Referencia oficial:** [pytest-cov Documentation](https://pytest-cov.readthedocs.io/)

In [ ]:
# Example: Running pytest with coverage
# Command: pytest --cov=mymodule --cov-report=html

def divide(a: float, b: float) -> float:
    """
    Divide two numbers.
    
    :param a: Numerator
    :type a: float
    :param b: Denominator
    :type b: float
    :return: Result of division
    :rtype: float
    :raises ZeroDivisionError: If b is zero
    """
    if b == 0:
        raise ZeroDivisionError("Cannot divide by zero")
    return a / b

# Test with incomplete coverage
def test_divide_normal():
    assert divide(10, 2) == 5.0
    # This test doesn't cover the ZeroDivisionError branch!
    # Coverage would show the error handling line as uncovered

# Complete coverage requires testing all branches
def test_divide_by_zero():
    with pytest.raises(ZeroDivisionError):
        divide(10, 0)
    # Now we have 100% coverage

### Interpretando Reportes de Cobertura

```
Name                Stmts   Miss  Cover
---------------------------------------
mymodule.py            10      2    80%
---------------------------------------
TOTAL                  10      2    80%
```

- **Stmts**: Número total de líneas ejecutables
- **Miss**: Líneas no ejecutadas por los tests
- **Cover**: Porcentaje de cobertura

### Aprendizaje Clave

Apunta a una cobertura alta (>80%), pero enfócate en la calidad de los tests. Es mejor tener 70% de cobertura con tests significativos que 100% con tests triviales.

**Referencia oficial:** [Coverage.py Documentation](https://coverage.readthedocs.io/)

## 8. Organizando Tests

### Estructura de Directorios

Una buena organización de tests facilita el mantenimiento y la navegación:

```
project/
├── src/
│   ├── __init__.py
│   ├── calculator.py
│   └── utils.py
└── tests/
    ├── __init__.py
    ├── conftest.py          # Fixtures compartidas
    ├── test_calculator.py
    └── test_utils.py
```

### conftest.py

El archivo `conftest.py` es especial en pytest. Las fixtures definidas aquí están disponibles automáticamente para todos los tests en ese directorio y subdirectorios.

### Aprendizaje Clave

Usa `conftest.py` para fixtures compartidas y configuración común. Mantén los tests organizados en archivos que reflejen la estructura de tu código fuente.

**Referencia oficial:** [pytest: conftest.py](https://docs.pytest.org/en/stable/reference/fixtures.html#conftest-py-sharing-fixtures-across-multiple-files)

In [ ]:
# Example conftest.py content
# This would be in tests/conftest.py

import pytest

@pytest.fixture(scope="session")
def test_config():
    """
    Provide test configuration.
    Available to all tests in the project.
    """
    return {
        "api_url": "https://test.api.example.com",
        "timeout": 30,
        "debug": True
    }

@pytest.fixture
def sample_data():
    """
    Provide sample data for tests.
    """
    return [
        {"id": 1, "name": "Alice", "age": 30},
        {"id": 2, "name": "Bob", "age": 25},
        {"id": 3, "name": "Charlie", "age": 35}
    ]

# Any test file can now use these fixtures
def test_with_config(test_config):
    assert test_config["timeout"] == 30

def test_with_sample_data(sample_data):
    assert len(sample_data) == 3

## Ejercicios Prácticos

### Ejercicio 1: Tests Básicos

Escribe tests para la siguiente función de validación de email. Asegúrate de probar casos normales, edge cases y casos de error.

In [ ]:
import re

def validate_email(email: str) -> bool:
    """
    Validate an email address.
    
    :param email: Email address to validate
    :type email: str
    :return: True if valid, False otherwise
    :rtype: bool
    """
    if not email or not isinstance(email, str):
        return False
    
    pattern = r'^[a-zA-Z0-9._%+-]+@[a-zA-Z0-9.-]+\.[a-zA-Z]{2,}$'
    return bool(re.match(pattern, email))

# TODO: Escribe tests para validate_email
# Casos a probar:
# - Email válido normal
# - Email con caracteres especiales permitidos
# - Email sin @
# - Email sin dominio
# - String vacío
# - None como input

### Ejercicio 2: Fixtures

Crea fixtures para configurar y limpiar un directorio temporal de tests. Luego escribe tests que usen estas fixtures.

In [ ]:
import os
import shutil

class FileManager:
    """
    Manage files in a directory.
    """
    def __init__(self, base_dir: str):
        self.base_dir = base_dir
    
    def create_file(self, filename: str, content: str) -> str:
        """
        Create a file with content.
        
        :param filename: Name of the file
        :type filename: str
        :param content: File content
        :type content: str
        :return: Full path to created file
        :rtype: str
        """
        filepath = os.path.join(self.base_dir, filename)
        with open(filepath, 'w') as f:
            f.write(content)
        return filepath
    
    def list_files(self) -> list:
        """
        List all files in the directory.
        
        :return: List of filenames
        :rtype: list
        """
        return os.listdir(self.base_dir)

# TODO: Crea una fixture que:
# 1. Cree un directorio temporal
# 2. Proporcione un FileManager para ese directorio
# 3. Limpie el directorio después del test

# TODO: Escribe tests que usen la fixture para:
# - Crear un archivo y verificar que existe
# - Crear múltiples archivos y listarlos
# - Verificar que el directorio se limpia después del test

### Ejercicio 3: Mocking

Escribe tests con mocking para la siguiente función que consulta una API de clima.

In [ ]:
import requests

class WeatherService:
    """
    Service to fetch weather data.
    """
    def __init__(self, api_key: str):
        self.api_key = api_key
        self.base_url = "https://api.weather.com/v1"
    
    def get_temperature(self, city: str) -> float:
        """
        Get current temperature for a city.
        
        :param city: City name
        :type city: str
        :return: Temperature in Celsius
        :rtype: float
        :raises requests.HTTPError: If API request fails
        """
        url = f"{self.base_url}/current"
        params = {"city": city, "api_key": self.api_key}
        
        response = requests.get(url, params=params)
        response.raise_for_status()
        
        data = response.json()
        return data["temperature"]

# TODO: Escribe tests con mocking para:
# - Caso exitoso: API retorna temperatura correctamente
# - Caso de error: API retorna 404
# - Caso de error: API retorna JSON inválido
# - Verificar que se llama a la API con los parámetros correctos

## Resumen

En este notebook has aprendido los fundamentos del testing unitario con pytest:

1. **pytest simplifica el testing** con aserciones de Python estándar y descubrimiento automático de tests

2. **Las fixtures eliminan duplicación** proporcionando configuración reutilizable con setup y teardown automático

3. **El mocking aísla dependencias** permitiendo tests rápidos, confiables y deterministas sin llamadas a sistemas externos

4. **La cobertura de código identifica gaps** pero la calidad de los tests es más importante que el porcentaje de cobertura

5. **La organización importa** usando `conftest.py` para fixtures compartidas y estructurando tests de forma clara

El testing unitario es una inversión que paga dividendos en confianza, velocidad de desarrollo y calidad de código. Los tests bien escritos son documentación ejecutable que demuestra cómo usar tu código.

## Preguntas de Autoevaluación

### 1. ¿Cuál es la principal ventaja de pytest sobre unittest?

**Respuesta:** pytest usa aserciones de Python estándar (`assert`) en lugar de métodos especiales (`assertEqual`, `assertTrue`, etc.), lo que hace el código más simple y legible. Además, pytest descubre tests automáticamente sin necesidad de heredar de clases especiales.

### 2. ¿Qué es el patrón AAA en testing y por qué es útil?

**Respuesta:** AAA significa Arrange-Act-Assert. Es un patrón que estructura los tests en tres fases: configurar el estado (Arrange), ejecutar la acción (Act), y verificar el resultado (Assert). Esto hace los tests más legibles y fáciles de entender.

### 3. ¿Cuándo deberías usar una fixture con scope="module" en lugar de scope="function"?

**Respuesta:** Usa `scope="module"` cuando la fixture es costosa de crear (como una conexión a base de datos) y puede ser compartida de forma segura entre múltiples tests sin causar interferencia. Usa `scope="function"` (default) cuando cada test necesita un estado limpio e independiente.

### 4. ¿Por qué es importante usar mocking al probar código que hace llamadas HTTP?

**Respuesta:** El mocking evita hacer llamadas HTTP reales, lo que hace los tests más rápidos, confiables y deterministas. Sin mocking, los tests dependerían de la red, serían lentos, podrían fallar por problemas de conectividad, y podrían tener efectos secundarios no deseados.

### 5. ¿Una cobertura de código del 100% garantiza que tu código está libre de bugs?

**Respuesta:** No. La cobertura del 100% solo significa que todas las líneas fueron ejecutadas, no que todos los casos posibles fueron probados ni que los tests son de calidad. Puedes tener 100% de cobertura con tests triviales que no verifican el comportamiento correcto.

### 6. ¿Qué hace el archivo conftest.py en pytest?

**Respuesta:** `conftest.py` es un archivo especial donde defines fixtures y configuración que se comparten automáticamente con todos los tests en ese directorio y subdirectorios. Es el lugar ideal para fixtures comunes y configuración de tests.

### 7. ¿Cuál es la diferencia entre usar `return` y `yield` en una fixture?

**Respuesta:** `return` solo proporciona el valor al test. `yield` permite separar setup (código antes de yield) y teardown (código después de yield), garantizando que el código de limpieza se ejecute incluso si el test falla.

## Recursos y Referencias Oficiales

### Documentación Oficial
- **[pytest Documentation](https://docs.pytest.org/en/stable/)**: Documentación completa de pytest con guías, tutoriales y referencia de API
- **[unittest.mock Documentation](https://docs.python.org/3/library/unittest.mock.html)**: Documentación oficial de la biblioteca de mocking de Python
- **[pytest-cov Documentation](https://pytest-cov.readthedocs.io/)**: Plugin de pytest para medir cobertura de código
- **[Coverage.py Documentation](https://coverage.readthedocs.io/)**: Herramienta subyacente para medir cobertura de código

### Guías y Mejores Prácticas
- **[pytest: How to use fixtures](https://docs.pytest.org/en/stable/how-to/fixtures.html)**: Guía completa sobre fixtures en pytest
- **[pytest: How to write and report assertions](https://docs.pytest.org/en/stable/how-to/assert.html)**: Guía sobre aserciones y mensajes de error
- **[pytest: Good Integration Practices](https://docs.pytest.org/en/stable/explanation/goodpractices.html)**: Mejores prácticas para organizar y estructurar tests

### Herramientas Relacionadas
- **[pytest-mock](https://pytest-mock.readthedocs.io/)**: Plugin que proporciona una fixture `mocker` para simplificar el uso de mocks
- **[pytest-xdist](https://pytest-xdist.readthedocs.io/)**: Plugin para ejecutar tests en paralelo
- **[hypothesis](https://hypothesis.readthedocs.io/)**: Biblioteca para property-based testing que se integra con pytest

### Notas Importantes
- Todos los enlaces están actualizados a partir de 2025
- Se recomienda revisar la documentación oficial regularmente para nuevas características
- pytest se actualiza frecuentemente con mejoras y nuevas funcionalidades